In [ ]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    image_files = os.listdir(image_directory)
    if not image_files:
        raise FileNotFoundError(f"No images found in directory: {image_directory}")
    
    example_image_path = os.path.join(image_directory, image_files[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = int(os.path.basename(file_path).split('_')[-1].split('.')[0])
    
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                try:
                    track_id = data[-1]
                    bbox_data = data[1:5]  # Ignore the first column
                    
                    x_center, y_center, width, height = map(float, bbox_data)
                
                    # Convert center coordinates to top-left coordinates
                    x_top_left = x_center - (width / 2)
                    y_top_left = y_center - (height / 2)
                    
                    output_line = f"{frame_number},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                    output_data.append(output_line)
                except ValueError as e:
                    print(f"Error processing line in file {file_path}: {line} -> {e}")
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    try:
        # Get the dimensions of one image
        image_width, image_height = get_image_dimensions(image_directory)
    except FileNotFoundError as e:
        print(e)
        return
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    if not os.path.exists(label_directory):
        print(f"Label directory does not exist: {label_directory}")
        return
    
    files = [file_name for file_name in os.listdir(label_directory) if file_name.endswith('.txt')]
    if not files:
        print(f"No label files found in directory: {label_directory}")
        return
    
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    if not all_data:
        print(f"No data to write for video: {video_name}")
        return
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['ocsort', 'deepocsort']
    
    for tracker in trackers:
        # from 0.2 to 0.8 with 0.05 step
        for iou in [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]:
        
            result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25__IoU_{iou}", 'data')
            output_dir = os.path.join(output_base_dir, f"{tracker}__input_1280__conf_.25__IoU_{iou}", "data")
            
            # Ensure the output directory exists
            os.makedirs(output_dir, exist_ok=True)
            
            sequences = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
            
            for seq in tqdm(sequences, desc=f"Converting seq for {tracker}", unit="video"):
                video_name = seq
                input_directory = os.path.join(result_dir, video_name)

                process_video(video_name, input_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/media/hbai/data/code/LiteSORT/datasets/DanceTrack/train'
result_base_dir = '/media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_IoU'
output_base_dir = '/media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


In [1]:
import os
import re
from tqdm import tqdm

def process_file(file_path):
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split(',')
            if len(data) == 10:
                try:
                    frame_number, track_id, x_center, y_center, width, height = map(float, data[:6])
                    
                    # Convert center coordinates to top-left coordinates
                    x_top_left = x_center - (width / 2)
                    y_top_left = y_center - (height / 2)
                    
                    output_line = f"{int(frame_number)},{int(track_id)},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                    output_data.append(output_line)
                except ValueError as e:
                    print(f"Error processing line in file {file_path}: {line} -> {e}")
    
    return output_data

def process_video(sequence_file, input_directory, output_directory):
    input_file = os.path.join(input_directory, sequence_file)
    if not os.path.exists(input_file):
        print(f"Input file does not exist: {input_file}")
        return
    
    output_file = os.path.join(output_directory, sequence_file)
    
    output_data = process_file(input_file)  # Process the entire sequence file
    if not output_data:
        print(f"No data processed for file: {input_file}")
        return
    
    with open(output_file, 'w') as out_file:
        for line in output_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(result_base_dir, output_base_dir):
    trackers = ['ocsort', 'deepocsort']
    
    for tracker in trackers:
        # from 0.2 to 0.8 with 0.05 step
        for iou in [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]:
        
            result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25__IoU_{iou}", 'data')
            output_dir = os.path.join(output_base_dir, f"{tracker}__input_1280__conf_.25__IoU_{iou}", "data")
            
            # Ensure the output directory exists
            os.makedirs(output_dir, exist_ok=True)
            
            sequence_files = [f for f in os.listdir(result_dir) if f.endswith('.txt')]
            
            for sequence_file in tqdm(sequence_files, desc=f"Converting seq for {tracker}", unit="file"):
                process_video(sequence_file, result_dir, output_dir)

# Specify the base result directory, and base output directory
result_base_dir = '/media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_IoU'
output_base_dir = '/media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format'

# Run the processing for all sequences for all trackers
main(result_base_dir, output_base_dir)


Converting seq for ocsort:  50%|█████     | 20/40 [00:00<00:00, 93.42file/s]

Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/ocsort__input_1280__conf_.25__IoU_0.2/data/dancetrack0023.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/ocsort__input_1280__conf_.25__IoU_0.2/data/dancetrack0096.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/ocsort__input_1280__conf_.25__IoU_0.2/data/dancetrack0062.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/ocsort__input_1280__conf_.25__IoU_0.2/data/dancetrack0098.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/ocsort__input_1280__conf_.25__IoU_0.2/data/dancetrack0051.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/ocsort__inpu

Converting seq for ocsort: 100%|██████████| 40/40 [00:00<00:00, 100.61file/s]


Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/ocsort__input_1280__conf_.25__IoU_0.2/data/dancetrack0053.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/ocsort__input_1280__conf_.25__IoU_0.2/data/dancetrack0086.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/ocsort__input_1280__conf_.25__IoU_0.2/data/dancetrack0083.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/ocsort__input_1280__conf_.25__IoU_0.2/data/dancetrack0027.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/ocsort__input_1280__conf_.25__IoU_0.2/data/dancetrack0061.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/ocsort__inpu

Converting seq for ocsort:   0%|          | 0/40 [00:00<?, ?file/s]

...

Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/deepocsort__input_1280__conf_.25__IoU_0.8/data/dancetrack0087.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/deepocsort__input_1280__conf_.25__IoU_0.8/data/dancetrack0039.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/deepocsort__input_1280__conf_.25__IoU_0.8/data/dancetrack0020.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/deepocsort__input_1280__conf_.25__IoU_0.8/data/dancetrack0082.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/deepocsort__input_1280__conf_.25__IoU_0.8/data/dancetrack0012.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed

Converting seq for deepocsort:  65%|██████▌   | 26/40 [00:00<00:00, 128.37file/s]

Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/deepocsort__input_1280__conf_.25__IoU_0.8/data/dancetrack0086.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/deepocsort__input_1280__conf_.25__IoU_0.8/data/dancetrack0083.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/deepocsort__input_1280__conf_.25__IoU_0.8/data/dancetrack0027.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/deepocsort__input_1280__conf_.25__IoU_0.8/data/dancetrack0061.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/deepocsort__input_1280__conf_.25__IoU_0.8/data/dancetrack0068.txt
Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed

Converting seq for deepocsort: 100%|██████████| 40/40 [00:00<00:00, 131.06file/s]

Data has been written to /media/hbai/data/code/LiteSORT/yolo_tracking/hbai_scripts/DanceTrack-train_fixed_format/deepocsort__input_1280__conf_.25__IoU_0.8/data/dancetrack0015.txt
